# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [1]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        
        for word in words:
            print("%s\t%d" % (word.lower(), 1))

    except ValueError as e:
        continue

Overwriting mapper1.py


In [2]:
%%writefile reducer1.py

import sys

current_word = None
current_count = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
        
        if current_word != key:
            if current_word:
                print("%s\t%d" % (current_word, current_count))
            
            current_word = key
            current_count = 0
        
        current_count += count
        
    except ValueError as e:
        continue
        
if current_word:       
    print("%s\t%d" % (current_word, current_count))

Overwriting reducer1.py


In [3]:
# You can use this cell for other experiments: for example, for combiner.

%%writefile combiner.py

import sys
current_word = None
current_count = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t',1)
        count = int(count)
        
        if current_word != key:
            if current_word:
                print( "%s\t%d"%(current_word, current_count) )
            current_word = key
            current_count = 0
        current_count += count
    except ValueError as e:
        continue
if current_word:
    print( "%s\t%d"%(current_word, current_count) )

## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

In [4]:
%%writefile mapper2.py

import sys

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
        print("{0}\t{1}".format(count, word))
    except ValueError as e:
        continue

Overwriting mapper2.py


In [5]:
%%writefile reducer2.py

import sys

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
        print("{0}\t{1}".format(word, count))
    except ValueError as e:
        continue

Overwriting reducer2.py


## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

__NB__: Please, use a defined python major version (e.g. `python3 mappper.py` instead of `python mapper.py`)!

In [7]:
%%bash

OUT_DIR_1="assignment1_1_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR_1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python3 mapper1.py" \
    -reducer "python3 reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR_1} > /dev/null


OUT_DIR_2="assignment1_2_"$(date +"%s%6N")

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount 2" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1,2nr" \
    -files mapper2.py,reducer2.py \
    -mapper "python3 mapper2.py" \
    -reducer "python3 reducer2.py" \
    -numReduceTasks 1 \
    -input ${OUT_DIR_1} \
    -output ${OUT_DIR_2} > /dev/null

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR_2}/part-00000 | sed -n '7p;8q'
#hdfs dfs -rm -r -skipTrash ${OUT_DIR_1}* > /dev/null
#hdfs dfs -rm -r -skipTrash ${OUT_DIR_2}* > /dev/null

rm: `assignment1_1_1615908403601284': No such file or directory
21/03/16 15:26:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/03/16 15:26:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
21/03/16 15:26:46 INFO mapred.FileInputFormat: Total input files to process : 1
21/03/16 15:26:46 INFO mapreduce.JobSubmitter: number of splits:2
21/03/16 15:26:46 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
21/03/16 15:26:46 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1615902652854_0012
21/03/16 15:26:47 INFO conf.Configuration: resource-types.xml not found
21/03/16 15:26:47 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
21/03/16 15:26:47 INFO resource.ResourceUtils: Adding resource type - name = memory-mb, units = Mi, type = COUNTABLE
21/03/16 15:26:47 INFO resource.ResourceUtils: Adding resource type - name = v